In [11]:
import pandas as pd
import numpy as np
import re
import time
import emoji
import torch
import torch.nn.functional as F
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import warnings
warnings.filterwarnings("ignore")

# Tiền xử lý tạo tập dữ liệu cơ bản với các đặc trưng thống kê

In [12]:
def preprocess_dataframe(df):
    # Xử lý nội dung văn bản
    df['has_emoji'] = df['text'].fillna('').apply(lambda x: int(any(c in emoji.EMOJI_DATA for c in x)))
    df['has_caps_words'] = df['text'].str.contains(r'\b[A-Z]{2,}\b', na=False).astype(int)

    # Xử lý media
    df['media_type_list'] = df['media_type'].fillna('').str.lower().apply(lambda x: [m.strip() for m in x.split(',') if m.strip()])
    for media_type in ['photo', 'video', 'animated_gif']:
        df[f'has_{media_type}'] = df['media_type_list'].apply(lambda x: int(media_type in x))
    df['media_count'] = df['media_type_list'].apply(len)

    # Trích xuất thông tin thời gian
    df['created_at'] = pd.to_datetime(df['created_at'], errors='coerce')
    df['hour'] = df['created_at'].dt.hour
    df['day_of_week'] = df['created_at'].dt.dayofweek
    df['is_weekend'] = (df['day_of_week'] >= 5).astype(int)
    df['month'] = df['created_at'].dt.month

    # Thông tin người dùng
    df['is_influencer'] = (df['user_followers_count'] > 100000).astype(int)

    # Ép kiểu cột boolean
    for col in ['is_repost', 'is_reply', 'is_quote']:
        if col in df.columns:
            df[col] = df[col].astype(int)

    # Tạo nhãn đầu ra 
    like_bins   = [-1, 0, 52, 2443, 9389, float('inf')]
    reply_bins  = [-1, 0, 2, 49, 228, float('inf')]
    quote_bins  = [-1, 0, 21, 107, 1151, float('inf')]
    repost_bins = [-1, 0, 91, 1087, 3823, float('inf')]

    labels = ['None', 'Very Low', 'Low', 'Medium', 'High']

    df['Like_Level']   = pd.cut(df['like_count'],   bins=like_bins,   labels=labels, include_lowest=True)
    df['Reply_Level']  = pd.cut(df['reply_count'],  bins=reply_bins,  labels=labels, include_lowest=True)
    df['Quote_Level']  = pd.cut(df['quote_count'],  bins=quote_bins,  labels=labels, include_lowest=True)
    df['Repost_Level'] = pd.cut(df['repost_count'], bins=repost_bins, labels=labels, include_lowest=True)

    # Xóa cột
    drop_cols = [
        'created_at', 'media_type', 'media_path', 'media_type_list',
        'post_id', 'text', 'like_count', 'repost_count',
        'reply_count', 'quote_count', 'cleaned_text'
    ]
    df.drop(columns=[col for col in drop_cols if col in df.columns], inplace=True)

    return df

if __name__ == '__main__':
    start_time = time.time()

    df = pd.read_csv('data.csv')
    df_processed = preprocess_dataframe(df)
    df_processed.to_csv('data_base.csv', index=False, encoding='utf-8')

    print(f'Thời gian xử lý: {time.time() - start_time:.2f} giây')
    print(f'Số dòng: {df_processed.shape[0]}')
    print(f'Số cột: {df_processed.shape[1]}')

Thời gian xử lý: 0.39 giây
Số dòng: 18064
Số cột: 25


# Tiền xử lý tạo tập dữ liệu mở rộng bao gồm các đặc trưng thống kê và các đặc trưng trích xuất từ nội dung post (sentiment, CTA, clickbait, NER)

In [13]:
# Làm sạch text
def clean_text(text):
    if pd.isna(text):
        return ''
    
    text = str(text).strip()

    # Xóa "RT" và người dùng repost
    text = re.sub(r'^RT @\w+:\s*|\bRT\b', '', text, flags=re.IGNORECASE)

    # Thay URL bằng [URL]
    text = re.sub(r'https?://\S+|www\.\S+', '[URL]', text)

    # Loại bỏ @ nhưng giữ tên người dùng
    text = re.sub(r'@(\w+)', r'\1', text)

    # Loại bỏ # nhưng giữ tên hashtag
    text = re.sub(r'#(\w+)', r'\1', text)

    # Xóa các thực thể HTML (như &amp;)
    text = re.sub(r'&[a-zA-Z0-9]+;', ' ', text)

    # Chuyển emoji sang dạng text (:heart:, :fire:)
    text = emoji.demojize(text, delimiters=(':', ':'))

    # Thêm khoảng trắng giữa emoji liền nhau
    text = re.sub(r'(:[^:\s]+:)(?=:)', r'\1 ', text)

    # Loại dấu : và _ (để dễ xử lý văn bản sau)
    text = re.sub(r'[:_]', ' ', text)

    # Xóa dấu câu và lặp dấu
    text = re.sub(r'[!?.,;]+', ' ', text)

    # Xóa ký tự đặc biệt còn lại
    text = re.sub(r'[^\w\s]', '', text)

    # Xóa dòng và chuẩn hóa khoảng trắng
    text = text.replace('\n', ' ').replace('\r', ' ')
    text = re.sub(r'\s+', ' ', text)

    return text.strip()

# Tải mô hình
if not torch.cuda.is_available():
    raise RuntimeError('Không tìm thấy GPU!')

DEVICE = torch.device('cuda')

# Mô hình phân tích cảm xúc
SENTIMENT_MODEL = 'cardiffnlp/twitter-xlm-roberta-base-sentiment'
tokenizer_sent = AutoTokenizer.from_pretrained(SENTIMENT_MODEL)
model_sent = AutoModelForSequenceClassification.from_pretrained(SENTIMENT_MODEL).to(DEVICE)

# Mô hình phân loại CTA & clickbait
cta_classifier = pipeline(
    'zero-shot-classification',
    model='joeddav/xlm-roberta-large-xnli',
    device=0 if torch.cuda.is_available() else -1
)

# Mô hình nhận diện thực thể
ner_pipe = pipeline(
    'ner',
    model='Davlan/xlm-roberta-base-ner-hrl',
    aggregation_strategy='simple',
    device=0 if torch.cuda.is_available() else -1
)

# Hàm chạy mô hình
def named_entit(texts, batch_size=32):
    flags = []
    for i in tqdm(range(0, len(texts), batch_size), desc='Phát hiện thực thể'):
        batch = texts[i:i + batch_size]
        try:
            results = ner_pipe(batch)
            flags.extend([int(len(res) > 0) for res in results])
        except:
            flags.extend([0] * len(batch))
    return flags

def sentiment_score(texts, batch_size=32):
    results = []
    model_sent.eval()

    for i in tqdm(range(0, len(texts), batch_size), desc='Tính điểm cảm xúc'):
        batch = [str(t) if t else '' for t in texts[i:i + batch_size]]
        inputs = tokenizer_sent(batch, padding=True, truncation=True, max_length=512, return_tensors='pt').to(DEVICE)
        
        with torch.no_grad():
            logits = model_sent(**inputs).logits
            probs = F.softmax(logits, dim=1).cpu().numpy()

        for prob in probs:
            score = prob[2] - prob[0]
            results.append(score)
    return results

def cta_clickbait(texts, threshold=0.5, batch_size=32):
    labels = ['contains CTA', 'clickbait']
    has_cta_flags = []
    is_clickbait_flags = []

    for i in tqdm(range(0, len(texts), batch_size), desc='Phân loại CTA / Clickbait'):
        batch = texts[i:i + batch_size]
        results = cta_classifier(batch, candidate_labels=labels, hypothesis_template='This text is {}.', multi_label=True)

        for res in results:
            scores = dict(zip(res['labels'], res['scores']))
            has_cta_flags.append(int(scores['contains CTA'] >= threshold))
            is_clickbait_flags.append(int(scores['clickbait'] >= threshold))
    
    return has_cta_flags, is_clickbait_flags

def preprocess_dataframe(df):
    # Làm sạch văn bản
    df['cleaned_text'] = df['text'].apply(clean_text)
    texts = df['text'].astype(str).tolist()
    texts_cleaned = df['cleaned_text'].tolist()

    # Đặc trưng từ văn bản
    df['has_emoji'] = df['text'].fillna('').apply(lambda x: int(any(c in emoji.EMOJI_DATA for c in x)))
    df['has_caps_words'] = df['text'].str.contains(r'\b[A-Z]{2,}\b', na=False).astype(int)

    # Phân tích media
    df['media_path_list'] = df['media_path'].fillna('').apply(lambda x: [p.strip() for p in x.split(',') if p.strip()])
    df['media_type_list'] = df['media_type'].fillna('').str.lower().apply(lambda x: [m.strip() for m in x.split(',') if m.strip()])
    for media_type in ['photo', 'video', 'animated_gif']:
        df[f'has_{media_type}'] = df['media_type_list'].apply(lambda x: int(media_type in x))
    df['media_count'] = df['media_type_list'].apply(len)

    # Thời gian
    df['created_at'] = pd.to_datetime(df['created_at'], errors='coerce')
    df['hour'] = df['created_at'].dt.hour
    df['day_of_week'] = df['created_at'].dt.dayofweek
    df['is_weekend'] = (df['day_of_week'] >= 5).astype(int)
    df['month'] = df['created_at'].dt.month

    # Người dùng
    df['is_influencer'] = (df['user_followers_count'] > 100_000).astype(int)

    # Chuyển cột boolean
    for col in ['is_repost', 'is_reply', 'is_quote']:
        if col in df.columns:
            df[col] = df[col].astype(int)

    # Phân tích ngữ nghĩa
    df['sentiment_score'] = sentiment_score(texts_cleaned)
    df['has_cta'], df['is_clickbait'] = cta_clickbait(texts)
    df['has_entities'] = named_entit(texts_cleaned)

    # Tạo nhãn đầu ra 
    like_bins   = [-1, 0, 52, 2443, 9389, float('inf')]
    reply_bins  = [-1, 0, 2, 49, 228, float('inf')]
    quote_bins  = [-1, 0, 21, 107, 1151, float('inf')]
    repost_bins = [-1, 0, 91, 1087, 3823, float('inf')]

    labels = ['None', 'Very Low', 'Low', 'Medium', 'High']

    df['Like_Level']   = pd.cut(df['like_count'],   bins=like_bins,   labels=labels, include_lowest=True)
    df['Reply_Level']  = pd.cut(df['reply_count'],  bins=reply_bins,  labels=labels, include_lowest=True)
    df['Quote_Level']  = pd.cut(df['quote_count'],  bins=quote_bins,  labels=labels, include_lowest=True)
    df['Repost_Level'] = pd.cut(df['repost_count'], bins=repost_bins, labels=labels, include_lowest=True)

    # Xoá cột không cần thiết
    cols_to_drop = [
        'created_at', 'media_type', 'media_path', 'media_type_list',
        'post_id', 'text', 'like_count', 'repost_count',
        'reply_count', 'quote_count'
    ]
    df.drop(columns=[c for c in cols_to_drop if c in df.columns], inplace=True)

    return df

if __name__ == '__main__':
    start_time = time.time()

    df = pd.read_csv('data.csv')
    df_processed = preprocess_dataframe(df)
    df_processed.to_csv('data_full.csv', index=False, encoding='utf-8')

    print(f'Thời gian xử lý: {time.time() - start_time:.2f} giây')
    print(f'Số dòng: {df_processed.shape[0]}')
    print(f'Số cột: {df_processed.shape[1]}')

Some weights of the model checkpoint at joeddav/xlm-roberta-large-xnli were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Phát hiện thực thể: 100%|██████████| 565/565 [45:05<00:00,  4.79s/it]


Thời gian xử lý: 11097.31 giây
Số dòng: 18064
Số cột: 31
